In [9]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_curve, auc
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.combine import SMOTEENN
from collections import Counter

In [76]:
df = pd.read_csv(r"C:\Users\moatz\Desktop\rental_info.csv")

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,moatz
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,1
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,1
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,1
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,1
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15856,2005-08-22 10:49:15+00:00,2005-08-29 09:52:15+00:00,6.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,48.8601,7744.0,24.9001,1
15857,2005-07-31 09:48:49+00:00,2005-08-04 10:53:49+00:00,4.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,24.9001,7744.0,24.9001,1
15858,2005-08-20 10:35:30+00:00,2005-08-29 13:03:30+00:00,8.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,80.8201,7744.0,24.9001,1
15859,2005-07-31 13:10:20+00:00,2005-08-08 14:07:20+00:00,7.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,63.8401,7744.0,24.9001,1


In [12]:
df['return_date'] = pd.to_datetime(df['return_date'])
df['rental_date'] = pd.to_datetime(df['rental_date'])
df['rental_length_days']=df['return_date']-df['rental_date'] 
df['rental_length_days']=df['return_date']-df['rental_date'] 
df['rental_length_days']=df['rental_length_days'].dt.days

In [13]:
df["deleted_scenes"] =  np.where(df["special_features"].str.contains("Deleted Scenes"), 1, 0)
# Add dummy for behind the scenes
df["behind_the_scenes"] =  np.where(df["special_features"].str.contains("Behind the Scenes"), 1, 0)

In [14]:
df.drop(['rental_date', 'return_date', 'special_features'], axis="columns", inplace=True)

In [15]:
df

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days,deleted_scenes,behind_the_scenes
0,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,3,0,1
1,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,2,0,1
2,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,7,0,1
3,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,2,0,1
4,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15856,6.99,2009.0,4.99,88.0,11.99,0,0,0,1,48.8601,7744.0,24.9001,6,1,1
15857,4.99,2009.0,4.99,88.0,11.99,0,0,0,1,24.9001,7744.0,24.9001,4,1,1
15858,8.99,2009.0,4.99,88.0,11.99,0,0,0,1,80.8201,7744.0,24.9001,9,1,1
15859,7.99,2009.0,4.99,88.0,11.99,0,0,0,1,63.8401,7744.0,24.9001,8,1,1


In [16]:
X=df.drop(['rental_length_days'],axis=1)
y=df['rental_length_days']


In [18]:
# Import the scaling module
from sklearn.preprocessing import StandardScaler

# Create a scaler object
scaler = StandardScaler()

inputs_scaled=scaler.fit_transform(X)

In [21]:
# Import the module for the split
from sklearn.model_selection import train_test_split

# Split the variables with an 80-20 split and some random state
# To have the same split as mine, use random_state = 365
x_train, x_test, y_train, y_test = train_test_split(inputs_scaled, y, test_size=0.2, random_state=365)

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
model = LinearRegression()

# Train the model on the training data
model.fit(x_train, y_train)

# Make predictions on the test data
y_pred = model.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
r2 = r2_score(y_test, y_pred)  # R-squared score

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 2.916947941357664
R-squared: 0.5979775530635872


In [23]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model to the training data
rf.fit(x_train, y_train)

# Make predictions
y_pred = rf.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Mean Squared Error: 2.039200443841836
R-squared: 0.7189513255949491


In [24]:
# Create the ANN model
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping  # Import EarlyStopping

model = Sequential()

# Input layer and hidden layers
model = Sequential()
model.add(Dense(123, input_shape=(x_train.shape[1],), activation="relu",kernel_regularizer=regularizers.l2(0.01)))
BatchNormalization(),
model.add(Dense(56, activation="relu",kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(16, activation="relu"))  
model.add(Dense(8, activation="relu"))   
model.add(LeakyReLU(alpha=0.01))

# Output layer
model.add(Dense(1))  # Single output value (sales prediction)

# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae','mse'])

# Train the model


early_stopping = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    x_train,
    y_train,
    epochs=5000,
    batch_size=4,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

# Make predictions
y_pred = model.predict(x_test)

# Evaluate model performance (on test set)
mse = model.evaluate(x_test, y_test)  # Corrected variable name
print(f'Mean Squared Error: {mse}')

# Print predictions and actual values
print("Predicted Sales:", y_pred.flatten())
print("Actual Sales:", y_test.values)

C:\Users\moatz\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 4.4605 - mae: 1.6818 - mse: 4.4605
Epoch 2/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 2.5184 - mae: 1.2895 - mse: 2.5184
Epoch 3/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 2.4091 - mae: 1.2533 - mse: 2.4091
Epoch 4/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 2.3558 - mae: 1.2422 - mse: 2.3558
Epoch 5/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 2.3376 - mae: 1.2298 - mse: 2.3376
Epoch 6/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 2.2929 - mae: 1.2221 - mse: 2.2929
Epoch 7/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 2.2385 - mae: 1.2069 - mse: 2.2385
Epoch 8/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 2.2234 - mae: 1.1967 - mse: 2.2234
Epoch 9/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 2.1890 - mae: 1.1863 - mse: 2.1890
Epoch 10/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 2.1423 - mae: 1.1813 - mse: 2.1423
Epoch 11/200

3172/3172 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 1.6913 - mae: 0.9959 - mse: 1.6913
Epoch 124/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 1.6444 - mae: 0.9804 - mse: 1.6444
Epoch 125/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 1.6593 - mae: 0.9894 - mse: 1.6593
Epoch 126/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 1.6210 - mae: 0.9733 - mse: 1.6210
Epoch 127/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 1.6297 - mae: 0.9804 - mse: 1.6297
Epoch 128/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 1.6241 - mae: 0.9783 - mse: 1.6241
Epoch 129/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 1.6519 - mae: 0.9826 - mse: 1.6519
Epoch 130/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 1.6007 - mae: 0.9672 - mse: 1.6007
Epoch 131/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 1.6499 - mae: 0.9878 - mse: 1.6499
Epoch 132/200
3172/3172 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 1.5979 - mae: 0.9746 - mse: 1.5979
Epoch 133/200

In [67]:
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")



R-squared: 0.7326390743255615


In [ ]:
print("Predicted Sales:", y_pred.flatten())
print("Actual Sales:", y_test.values)

In [66]:
pd.DataFrame({'Predicted Sales': y_pred.flatten(),'Actual Sales':y_test.values})

,Predicted Sales,Actual Sales
0,2.957545,3
1,3.417367,0
2,4.586077,4
3,2.527959,6
4,7.933076,8
...,...,...
3168,2.390293,1
3169,7.357182,7
3170,4.618639,8
3171,4.280032,3


In [27]:
import pickle
with open(r'C:\Users\moatz\Desktop\New folder/model_pickle.pkl', 'wb') as file:
    pickle.dump(model, file)